<a href="https://colab.research.google.com/github/teamgaon/SANUP/blob/main/220401_sm_SANUP_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 라이브러리, 패키지

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 3.8 MB 58.2 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 80.0 MB/s 
     |████████████████████████████████| 1.1 MB 67.1 MB/s 
     |████████████████████████████████| 136 kB 82.5 MB/s 
     |████████████████████████████████| 127 kB 85.9 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
     |████████████████████████████████| 144 kB 77.6 MB/s 
     |████████████████████████████████| 271 kB 78.8 MB/s 
     |████████████████████████████████| 895 kB 64.7 MB/s 
     |████████████████████████████████| 596 kB 72.3 MB/s 
     |████████████████████████████████| 6.6 MB 51.9 MB/s 
     |████████████████████████████████| 1.2 MB 66.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

- TPU 사용을 위한 accelerate

In [2]:
from accelerate import Accelerator

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher

In [25]:
df = pd.read_csv('/content/drive/MyDrive/SANUP/1. 실습용자료.txt', sep='|', encoding='cp949')
test = pd.read_csv('/content/drive/MyDrive/SANUP/2. 모델개발용자료.txt', sep='|', encoding='cp949')
submission = pd.read_csv('/content/drive/MyDrive/SANUP/답안 작성용 파일.csv', encoding='cp949')

In [26]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [27]:
df = df.fillna(0)

In [28]:
df['text'] = df['text_obj'].astype(str) + " " + df['text_mthd'].astype(str) + " " + df['text_deal'].astype(str)
df['target'] = df['digit_1'] + " " + df['digit_2'].astype(str) + " " + df['digit_3'].astype(str)
df

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_0000001,S,95,952,카센터에서,자동차부분정비,타이어오일교환,카센터에서 자동차부분정비 타이어오일교환,S 95 952
1,id_0000002,G,47,472,상점내에서,일반인을 대상으로,채소.과일판매,상점내에서 일반인을 대상으로 채소.과일판매,G 47 472
2,id_0000003,G,46,467,절단하여사업체에도매,공업용고무를가지고,합성고무도매,절단하여사업체에도매 공업용고무를가지고 합성고무도매,G 46 467
3,id_0000004,G,47,475,영업점에서,일반소비자에게,열쇠잠금장치,영업점에서 일반소비자에게 열쇠잠금장치,G 47 475
4,id_0000005,Q,87,872,어린이집,보호자의 위탁을 받아,취학전아동보육,어린이집 보호자의 위탁을 받아 취학전아동보육,Q 87 872
...,...,...,...,...,...,...,...,...,...
999995,id_0999996,C,13,134,제품입고,워싱,청바지워싱,제품입고 워싱 청바지워싱,C 13 134
999996,id_0999997,F,42,424,현장에서,고객의요청에의해,실내인테리어,현장에서 고객의요청에의해 실내인테리어,F 42 424
999997,id_0999998,G,47,474,영업점에서,일반소비자에게,여성의류 판매,영업점에서 일반소비자에게 여성의류 판매,G 47 474
999998,id_0999999,P,85,856,사업장에서,일반고객을대상으로,필라테스,사업장에서 일반고객을대상으로 필라테스,P 85 856


In [29]:
train = df[['text', 'target']]
train

,text,target
0,카센터에서 자동차부분정비 타이어오일교환,S 95 952
1,상점내에서 일반인을 대상으로 채소.과일판매,G 47 472
2,절단하여사업체에도매 공업용고무를가지고 합성고무도매,G 46 467
3,영업점에서 일반소비자에게 열쇠잠금장치,G 47 475
4,어린이집 보호자의 위탁을 받아 취학전아동보육,Q 87 872
...,...,...
999995,제품입고 워싱 청바지워싱,C 13 134
999996,현장에서 고객의요청에의해 실내인테리어,F 42 424
999997,영업점에서 일반소비자에게 여성의류 판매,G 47 474
999998,사업장에서 일반고객을대상으로 필라테스,P 85 856


# 토크나이저, 함수

In [30]:
from transformers import AutoTokenizer
from transformers import ElectraModel, ElectraTokenizer

checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257k [00:00<?, ?B/s]

In [31]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [32]:
my_dict = {}
for i in range(len(train['target'].unique())):
  my_dict[train['target'].unique()[i]] = i

def target_to_num(target:str):
  return my_dict[target]

train['target'] = train['target'].map(target_to_num)
train

,text,target
0,카센터에서 자동차부분정비 타이어오일교환,0
1,상점내에서 일반인을 대상으로 채소.과일판매,1
2,절단하여사업체에도매 공업용고무를가지고 합성고무도매,2
3,영업점에서 일반소비자에게 열쇠잠금장치,3
4,어린이집 보호자의 위탁을 받아 취학전아동보육,4
...,...,...
999995,제품입고 워싱 청바지워싱,103
999996,현장에서 고객의요청에의해 실내인테리어,68
999997,영업점에서 일반소비자에게 여성의류 판매,11
999998,사업장에서 일반고객을대상으로 필라테스,12


In [33]:
my_dict['0.0 0.0 0.0'] = len(my_dict)

In [14]:
train[train['text'].isnull()]

,text,target


In [15]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# 학습

* Model
  + Roberta-large

* Optimizer
  + AdamW

* Learning rate scheduler
  + Cosine annealing with warmup

In [16]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=32, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=32, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = len(train['target'].unique())
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 10
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )

  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  unwrapped_model.save_pretrained('/content/drive/MyDrive/220331_inference/model' + str(fold), save_function=accelerator.save)

In [17]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['text']),
  return_tensors="pt",
  max_length=64, # Max_Length = 64
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['target'])):
  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = train['target']

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: 

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

...

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

  0%|          | 0/31250 [00:00<?, ?it/s]

epoch 1 train acc 0.70348
epoch 1 train acc 0.70721
epoch 1 train acc 0.7049
epoch 1 train acc 0.70514
epoch 1 train acc 0.70524
epoch 1 train acc 0.70537
epoch 1 train acc 0.70385
epoch 1 train acc 0.70433
epoch 1 test acc 0.8651294757033248
epoch 1 test acc 0.8700847186700768
epoch 1 test acc 0.8693654092071611
epoch 1 test acc 0.865888746803069
epoch 1 test acc 0.8708040281329923
epoch 1 test acc 0.8670875959079284
epoch 1 test acc 0.8707640664961637
epoch 1 test acc 0.8670076726342711
epoch 2 train acc 0.88131
epoch 2 train acc 0.88029
epoch 2 train acc 0.88023
epoch 2 train acc 0.879
epoch 2 train acc 0.87918
epoch 2 train acc 0.87917
epoch 2 train acc 0.88079
epoch 2 train acc 0.88085
epoch 2 test acc 0.900175831202046
epoch 2 test acc 0.9015744884910486
epoch 2 test acc 0.8980179028132992
epoch 2 test acc 0.8973385549872123
epoch 2 test acc 0.9026134910485933
epoch 2 test acc 0.8980179028132992
epoch 2 test acc 0.8994565217391305
epoch 2 test acc 0.8971787084398977
epoch 3 train

# 추론

In [34]:
test = test.fillna(0)

In [35]:
test['text'] = test['text_obj'].astype(str) + " " + test['text_mthd'].astype(str) + " " + test['text_deal'].astype(str)
test['target'] = test['digit_1'].astype(str) + " " + test['digit_2'].astype(str) + " " + test['digit_3'].astype(str)
test

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal,text,target
0,id_000001,0.0,0.0,0.0,치킨전문점에서,고객의주문에의해,치킨판매,치킨전문점에서 고객의주문에의해 치킨판매,0.0 0.0 0.0
1,id_000002,0.0,0.0,0.0,산업공구,다른 소매업자에게,철물 수공구,산업공구 다른 소매업자에게 철물 수공구,0.0 0.0 0.0
2,id_000003,0.0,0.0,0.0,절에서,신도을 대상으로,불교단체운영,절에서 신도을 대상으로 불교단체운영,0.0 0.0 0.0
3,id_000004,0.0,0.0,0.0,영업장에서,고객요구로,자동차튜닝,영업장에서 고객요구로 자동차튜닝,0.0 0.0 0.0
4,id_000005,0.0,0.0,0.0,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공","실내포장마차에서 접객시설을 갖추고 소주,맥주제공",0.0 0.0 0.0
...,...,...,...,...,...,...,...,...,...
99995,id_099996,0.0,0.0,0.0,사업장에서,일반인대상으로,버섯농장,사업장에서 일반인대상으로 버섯농장,0.0 0.0 0.0
99996,id_099997,0.0,0.0,0.0,한의원에서,외래환자위주고,치료,한의원에서 외래환자위주고 치료,0.0 0.0 0.0
99997,id_099998,0.0,0.0,0.0,일반점포에서,소비자에게,그림판매,일반점포에서 소비자에게 그림판매,0.0 0.0 0.0
99998,id_099999,0.0,0.0,0.0,사업장에서,일반인.학생대상으로,학습공간제공,사업장에서 일반인.학생대상으로 학습공간제공,0.0 0.0 0.0


In [36]:
test = test[['text', 'target']]
test

,text,target
0,치킨전문점에서 고객의주문에의해 치킨판매,0.0 0.0 0.0
1,산업공구 다른 소매업자에게 철물 수공구,0.0 0.0 0.0
2,절에서 신도을 대상으로 불교단체운영,0.0 0.0 0.0
3,영업장에서 고객요구로 자동차튜닝,0.0 0.0 0.0
4,"실내포장마차에서 접객시설을 갖추고 소주,맥주제공",0.0 0.0 0.0
...,...,...
99995,사업장에서 일반인대상으로 버섯농장,0.0 0.0 0.0
99996,한의원에서 외래환자위주고 치료,0.0 0.0 0.0
99997,일반점포에서 소비자에게 그림판매,0.0 0.0 0.0
99998,사업장에서 일반인.학생대상으로 학습공간제공,0.0 0.0 0.0


In [37]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

test_label = test['target'].map(target_to_num)

tokenized_test = tokenizer(
    list(test['text']),
    return_tensors="pt",
    max_length=64,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

for fold in range(5):

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 225
  model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/220331_inference/model' + str(fold), num_labels=225)
  model.resize_token_embeddings(tokenizer.vocab_size)
  accelerator = Accelerator()
  model = accelerator.unwrap_model(model)

  output_pred = []
  output_prob = []

  model, dataloader= accelerator.prepare(model, dataloader)

  model.eval()

  for i, data in enumerate(tqdm(dataloader)):
      with torch.no_grad():
          outputs = model(
              input_ids=data['input_ids'],
              attention_mask=data['attention_mask']
          )
      logits = outputs[0]
      prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
      logits = logits.detach().cpu().numpy()
      result = np.argmax(logits, axis=-1)
      output_pred.append(result)
      output_prob.append(prob)
      
  pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()

  def num_to_label(label):
      label_dict = {v: k for k, v in my_dict.items()}
      str_label = []

      for i, v in enumerate(label):
          str_label.append([i,label_dict[v]])
      
      return str_label

  answer = num_to_label(pred_answer)

  df_label = pd.DataFrame(answer, columns=['index', 'label'])
  df_prob = pd.DataFrame(output_prob)

  df_label.to_csv('/content/drive/MyDrive/220331_inference/pred_label'+str(fold)+'.csv', index=False)
  df_prob.to_csv('/content/drive/MyDrive/220331_inference/pred_prob'+str(fold)+'.csv', index=False)

KeyboardInterrupt: ignored

In [22]:
print(my_variable)

NameError: ignored

In [38]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [39]:
pred0 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_label4.csv')

In [40]:
test['label0'] = pred0['label']
test['label1'] = pred1['label']
test['label2'] = pred2['label']
test['label3'] = pred3['label']
test['label4'] = pred4['label']
df = test

In [41]:
df.head(3)

,text,target,label0,label1,label2,label3,label4
0,치킨전문점에서 고객의주문에의해 치킨판매,0.0 0.0 0.0,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561
1,산업공구 다른 소매업자에게 철물 수공구,0.0 0.0 0.0,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466
2,절에서 신도을 대상으로 불교단체운영,0.0 0.0 0.0,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949


In [42]:
1-((len(df)-len(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])]))/len(df))

0.9089700000000001

In [43]:
hard = df[df.merge(df[(df['label0'] == df['label1']) & (df['label1'] == df['label2']) & (df['label2'] == df['label3']) & (df['label3'] == df['label4'])], how = 'left', left_index=True, right_index=True)['text_y'].isnull()]
hard

,text,target,label0,label1,label2,label3,label4
17,"상가에서 주문에 의해 반찬,도시락",0.0 0.0 0.0,C 10 107,C 10 107,C 10 107,G 47 472,C 10 107
31,산업사용자에게 선별장에서 농산물도매,0.0 0.0 0.0,G 46 463,G 46 462,G 46 463,G 46 462,G 46 463
37,카드사등 G/W 기술 SMS 서비스,0.0 0.0 0.0,J 62 620,J 58 582,J 62 620,J 62 620,J 58 582
38,"PP.PET 원료투입, 압출 원사",0.0 0.0 0.0,C 13 139,C 13 139,C 13 139,C 13 139,C 22 222
49,매장에서 일반소비자에게 식료품소매1,0.0 0.0 0.0,G 47 471,G 47 471,G 47 472,G 47 472,G 47 471
...,...,...,...,...,...,...,...
99967,업장에서 환자에게 채활치료서비스,0.0 0.0 0.0,Q 86 869,Q 86 869,Q 86 861,Q 86 869,Q 86 869
99979,업주들을대상으로 사무실에서 검사및회계사무지원,0.0 0.0 0.0,M 71 712,M 71 712,M 71 712,M 71 715,M 71 712
99985,"철 구입, 기계절단 0",0.0 0.0 0.0,C 25 259,C 25 259,C 25 259,C 25 259,C 29 291
99989,사업장에서 일반소비자및 소매업자 계란소매,0.0 0.0 0.0,G 46 463,G 46 463,G 47 472,G 46 463,G 46 463


In [44]:
pred0 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob0.csv')
pred1 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob1.csv')
pred2 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob2.csv')
pred3 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob3.csv')
pred4 = pd.read_csv('/content/drive/MyDrive/220331_inference/pred_prob4.csv')

In [45]:
pred = pd.DataFrame((np.array(pred0) + np.array(pred1) + np.array(pred2) + np.array(pred3) + np.array(pred4))/5)
# test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')
test = pd.concat([test, pred], axis=1)
test.head(3)

,text,target,label0,label1,label2,label3,label4,0,1,2,...,215,216,217,218,219,220,221,222,223,224
0,치킨전문점에서 고객의주문에의해 치킨판매,0.0 0.0 0.0,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561,3.186903e-08,0.000293,2.241382e-07,...,2.811072e-07,1.119740e-07,1.638260e-08,3.115402e-09,1.227746e-09,5.218897e-09,9.465076e-09,4.863284e-09,9.318668e-10,4.629735e-09
1,산업공구 다른 소매업자에게 철물 수공구,0.0 0.0 0.0,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466,1.046507e-05,0.000002,2.093271e-03,...,1.480631e-07,3.368968e-08,1.691002e-06,7.235341e-08,7.895837e-09,1.911023e-08,3.236868e-08,1.482533e-07,1.926374e-08,3.113599e-08
2,절에서 신도을 대상으로 불교단체운영,0.0 0.0 0.0,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,2.610736e-09,0.000002,4.406631e-06,...,3.574516e-08,1.382377e-09,8.871879e-09,4.434426e-09,1.651850e-09,1.089021e-08,2.572904e-08,3.170224e-08,6.859331e-10,5.692115e-09


In [46]:
def num_to_label(label):
    label_dict = label_dict = {v: k for k, v in my_dict.items()}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(np.argmax(np.array(pred), axis=-1))

test['soft'] = pd.DataFrame(answer)[1]
test.head(3)

,text,target,label0,label1,label2,label3,label4,0,1,2,...,216,217,218,219,220,221,222,223,224,soft
0,치킨전문점에서 고객의주문에의해 치킨판매,0.0 0.0 0.0,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561,3.186903e-08,0.000293,2.241382e-07,...,1.119740e-07,1.638260e-08,3.115402e-09,1.227746e-09,5.218897e-09,9.465076e-09,4.863284e-09,9.318668e-10,4.629735e-09,I 56 561
1,산업공구 다른 소매업자에게 철물 수공구,0.0 0.0 0.0,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466,1.046507e-05,0.000002,2.093271e-03,...,3.368968e-08,1.691002e-06,7.235341e-08,7.895837e-09,1.911023e-08,3.236868e-08,1.482533e-07,1.926374e-08,3.113599e-08,G 46 466
2,절에서 신도을 대상으로 불교단체운영,0.0 0.0 0.0,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,2.610736e-09,0.000002,4.406631e-06,...,1.382377e-09,8.871879e-09,4.434426e-09,1.651850e-09,1.089021e-08,2.572904e-08,3.170224e-08,6.859331e-10,5.692115e-09,S 94 949


In [47]:
soft = test[['label0', 'label1', 'label2', 'label3', 'label4', 'soft']]
soft

,label0,label1,label2,label3,label4,soft
0,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561
1,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466
2,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949
3,S 95 952,S 95 952,S 95 952,S 95 952,S 95 952,S 95 952
4,I 56 562,I 56 562,I 56 562,I 56 562,I 56 562,I 56 562
...,...,...,...,...,...,...
99995,G 46 463,A 1 11,G 47 472,G 46 463,A 1 11,G 46 463
99996,Q 86 862,Q 86 862,Q 86 862,Q 86 862,Q 86 862,Q 86 862
99997,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478
99998,R 90 902,R 90 902,R 90 902,R 90 902,R 90 902,R 90 902


In [48]:
def label_count(df):
  num_count = {}
  cols = ['label0', 'label1', 'label2', 'label3', 'label4']

  for col in cols:
    try: num_count[df[col]] += 1
    except: num_count[df[col]] = 1
  return(num_count)

temp = soft.apply(label_count, axis=1)
soft['temp'] = temp

def label_count(temp:dict):
  keys = []
  temptemp = 0
  for k, v in temp.items():
    if v >= 3:
      temptemp = k
  keys.append(temptemp)
  return keys[0]
  
soft['hard'] = soft['temp'].map(label_count)

In [49]:
soft[soft['soft'] != soft['hard']]

,label0,label1,label2,label3,label4,soft,temp,hard
119,M 70 702,R 90 901,M 70 701,M 70 701,R 90 901,R 90 901,"{'M 70 702': 1, 'R 90 901': 2, 'M 70 701': 2}",0
287,P 85 856,R 91 911,P 85 856,P 85 856,R 91 911,R 91 911,"{'P 85 856': 3, 'R 91 911': 2}",P 85 856
322,C 29 292,C 29 291,C 29 292,C 29 291,C 29 292,C 29 291,"{'C 29 292': 3, 'C 29 291': 2}",C 29 292
328,M 71 711,N 75 759,N 75 759,N 75 759,M 71 711,M 71 711,"{'M 71 711': 2, 'N 75 759': 3}",N 75 759
359,J 63 639,J 63 639,J 62 620,J 63 639,J 62 620,J 62 620,"{'J 63 639': 3, 'J 62 620': 2}",J 63 639
...,...,...,...,...,...,...,...,...
99815,G 46 467,G 46 467,G 46 465,G 46 467,G 46 465,G 46 465,"{'G 46 467': 3, 'G 46 465': 2}",G 46 467
99838,C 10 107,C 10 106,C 10 107,C 10 106,C 10 106,C 10 107,"{'C 10 107': 2, 'C 10 106': 3}",C 10 106
99847,C 14 144,C 14 144,C 13 134,C 14 143,C 13 131,C 13 134,"{'C 14 144': 2, 'C 13 134': 1, 'C 14 143': 1, ...",0
99933,S 94 941,S 94 941,S 94 941,S 94 949,S 94 949,S 94 949,"{'S 94 941': 3, 'S 94 949': 2}",S 94 941


In [50]:
df = soft[['label0',	'label1',	'label2',	'label3',	'label4',	'soft',	'hard']]
df

,label0,label1,label2,label3,label4,soft,hard
0,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561,I 56 561
1,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466,G 46 466
2,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949,S 94 949
3,S 95 952,S 95 952,S 95 952,S 95 952,S 95 952,S 95 952,S 95 952
4,I 56 562,I 56 562,I 56 562,I 56 562,I 56 562,I 56 562,I 56 562
...,...,...,...,...,...,...,...
99995,G 46 463,A 1 11,G 47 472,G 46 463,A 1 11,G 46 463,0
99996,Q 86 862,Q 86 862,Q 86 862,Q 86 862,Q 86 862,Q 86 862,Q 86 862
99997,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478,G 47 478
99998,R 90 902,R 90 902,R 90 902,R 90 902,R 90 902,R 90 902,R 90 902


In [51]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,NaN,NaN,NaN,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,NaN,NaN,NaN,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,NaN,NaN,NaN,절에서,신도을 대상으로,불교단체운영
3,id_000004,NaN,NaN,NaN,영업장에서,고객요구로,자동차튜닝
4,id_000005,NaN,NaN,NaN,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,NaN,NaN,NaN,사업장에서,일반인대상으로,버섯농장
99996,id_099997,NaN,NaN,NaN,한의원에서,외래환자위주고,치료
99997,id_099998,NaN,NaN,NaN,일반점포에서,소비자에게,그림판매
99998,id_099999,NaN,NaN,NaN,사업장에서,일반인.학생대상으로,학습공간제공


In [ ]:
print(my_variable)

In [52]:
def split_label(text:str):
  return text.split(' ')[0]

submission['digit_1'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[1]

submission['digit_2'] = df['soft'].map(split_label)

def split_label(text:str):
  return text.split(' ')[2]

submission['digit_3'] = df['soft'].map(split_label)

In [53]:
submission

,AI_id,digit_1,digit_2,digit_3,text_obj,text_mthd,text_deal
0,id_000001,I,56,561,치킨전문점에서,고객의주문에의해,치킨판매
1,id_000002,G,46,466,산업공구,다른 소매업자에게,철물 수공구
2,id_000003,S,94,949,절에서,신도을 대상으로,불교단체운영
3,id_000004,S,95,952,영업장에서,고객요구로,자동차튜닝
4,id_000005,I,56,562,실내포장마차에서,접객시설을 갖추고,"소주,맥주제공"
...,...,...,...,...,...,...,...
99995,id_099996,G,46,463,사업장에서,일반인대상으로,버섯농장
99996,id_099997,Q,86,862,한의원에서,외래환자위주고,치료
99997,id_099998,G,47,478,일반점포에서,소비자에게,그림판매
99998,id_099999,R,90,902,사업장에서,일반인.학생대상으로,학습공간제공


In [54]:
submission.to_csv('/content/drive/MyDrive/220331_inference/submission.csv', index=False)